In [ ]:
import findspark
findspark.init('/usr/local/spark')

In [8]:
from pyspark import SparkContext, SparkConf, SQLContext
import pandas as pd
import re
from operator import add
import numpy as np
import hashlib
from datasketch.minhash import MinHash
from datasketch.lsh import MinHashLSH
import csv

In [9]:
sc = SparkContext.getOrCreate()
questionFile=sc.textFile("Questions.csv")
answerFile=sc.textFile("Answers.csv")
tagFile = sc.textFile("Tags.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finished")

finished


In [ ]:
#Read sample of question.csv(5%), generate the keywords list to focus on

In [10]:
questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = questionWithHeader.first() #extract header
question = questionWithHeader.filter(lambda x: x != header)
print(question.count())

1264216


In [11]:
questionSample=question.sample(False,0.05,81)
questionLower = questionSample.map(lambda x: (x[0], re.sub("<.*?>", " ", (x[5]+x[6]).lower())))
def f(x): return re.sub('[^a-zA-Z0-9\']+', ' ', x)
questionLower = questionLower.mapValues(f)
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.count())

13348045


In [12]:
stopWordsList = stopWordsFile.collect()
questionPair=questionPairRaw.filter(lambda x: x[1] not in stopWordsList)
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
N = len(questionPair.countByKey())
print(N)

63340


In [11]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
stage2Map = questionPairStage2MapPhase.countByKey()
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
print(questionPairStage3AggByKey.count())

63340


In [12]:
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.count())
questionPairStage4.take(5)

63340


[('292860',
  ['esb=0.31955710438882234',
   'integration=0.26415616399604536',
   'jdbc=0.26063689583132227',
   'basis=0.2536044558888481',
   'redeploy=0.2316600941696921',
   'module=0.14232311356590852',
   'highly=0.13782171770001034']),
 ('432500',
  ['helptext=0.31347145520530423',
   'feature=0.3039788295804193',
   'sql=0.2814420402886694',
   'procedures=0.21630610982917833',
   'enterprise=0.2050126542510726',
   'studio=0.13212127154225234',
   'g=0']),
 ('435980',
  ['recompile=0.4738906693757042',
   'justifiable=0.3484750543457793',
   'declared=0.26399868358459483',
   'health=0.22104489800628596',
   'commented=0.1848018136332991',
   'find=0.1141603024729195',
   'sql=0.09390712143738686']),
 ('525430',
  ['gameconfig=0.39542916360950864',
   'getscreensize=0.21499596093759327',
   'getextendedstate=0.15153806178919574',
   'graphicsconfiguration=0.15153806178919574',
   'screenmaximized=0.15153806178919574',
   'windowy=0.14203773679396084',
   'windowx=0.1420377367

In [13]:
def fmap1(x):
    output=""
    for i in range(num_keywords):
        y=x[1][i].split("=")
        if float(y[1])>0.4:
           output+=y[0]+";"
    return ("key",output)
keywordlist=questionPairStage4.map(fmap1).reduceByKey(add)
keywords=keywordlist.take(1)
words=keywords[0][1][:-1]
keywordset=set(words.split(";"))
print(len(keywordset))
print(keywordset)

27911
{'duedate', 'differnce', 'ltsp', 'circle', 'deletebutton', 'review', 'solicitation', 'preceding', 'escript', 'mftpclient', 'outputting', 'gu', 'crop', 'objectxy', 'zqhh', 'formobject', 'dicom', 'toksykologia', 'queuelist', 'chang', 'updateimpression', 'pikaday', 'ischeck', 'dummydf', 'general', 'ffffcc', 'nsdate', 'rbind', 'parnew', 'autodate', 'aboutboxcontroller', 'previousrange', 'puppetmaster', 'nswaps', 'ddhash', 'disposing', 'coherent', 'gridgain', 'testtmpblock', 'filers', 'findone', 'usesubmitbeheviour', 'eminem', 'multifile', 'program', 'checkstyle', 'originating', 'tabletwo', 'customeremail', 'oldcolumn', 'qtcpsocket', 'grob', 'hashset', 'xclip', 'realsite', 'satisfied', 'numberstring', 'xmm', 'tibco', 'instdir', 'nodes', 'raw', 'lqci', 'soemone', 'attributed', 'xpzd', 'msgfilename', 'continent', 'axis', 'testconnection', 'mkv', 'makeres', 'documenticontext', 'genereal', 'friendbis', 'bobdole', 'changeimg', 'stripslash', 'hely', 'movdqa', 'httprequest', 'ltc', 'dakka', 

In [ ]:
#turn oringinal question pairs to pairs only keep the keywords,get 7 keywords for each question

In [15]:
questionLower = question.map(lambda x: (x[0], re.sub("<.*?>", " ", (x[5]+x[6]).lower())))
def f(x): return re.sub('[^a-zA-Z0-9\']+', ' ', x)
questionLower = questionLower.mapValues(f)
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
questionPair=questionPairRaw.filter(lambda x: x[1] in keywordset)
print(questionPair.count())

67819009


In [16]:
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
N = len(questionPair.countByKey())
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
stage2Map = questionPairStage2MapPhase.countByKey()
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.count())
print(questionPairStage2.take(5))

28565882
[('statement@80', 6.205236921573187), ('log@650', 2.7946709458828387), ('number@1160', 2.586606286929646), ('ordering@1610', 6.259204202716805), ('asax@2770', 7.168488930249008)]


In [17]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
print(stage3Map.get('90'))

20.311206869886846


In [18]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))

num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.take(5))

[('8443560', ['thrift=0.49115149799457625', 'cassandra=0.33217741418255603', 'codeplex=0.21631640020847592', 'protocol=0.17369330327033117', 'len=0.16851337078099782', 'host=0.1655789277138693', 'cluster=0.13966007860011193']), ('32109090', ['interacting=0.3553307011283048', 'viewcontroller=0.32994375674521975', 'pushnotification=0.2567311744813316', 'navigation=0.21832888292463984', 'notification=0.1391414077733566', 'depending=0.12887402255724206', 'g=0']), ('12657390', ['paragraph=0.5987292905603192', 'tutorial=0.3434025110838559', 'knowing=0.2798768873942756', 'format=0.24865331108614458', 'users=0.23617071563762812', 'f=0', 'g=0']), ('13198270', ['webmethod=0.28331548275147156', 'orgdata=0.2754159451684192', 'classname=0.2676531474462438', 'packagename=0.2533064402056847', 'javax=0.18742974309228122', 'integer=0.17559729242900116', 'executions=0.1615754361040162']), ('13409430', ['viewport=0.6842902201655965', 'floating=0.5905586792866232', 'component=0.24216553928531787', 'd=0', 

In [19]:
outMap=questionPairStage4.collectAsMap()
tagWithHeader = tagFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = tagWithHeader.first() #extract header
tag = tagWithHeader.filter(lambda x: x != header)
tagLower = tag.map(lambda x: (x[0], x[1].lower()))
def f0(x): return re.sub('[^a-zA-Z]+', '', x)
tagLower = tagLower.mapValues(f0)
tag_1000 = tagLower.map(lambda x: (x[0], x[1]+"=1000"))
questionPairStage5=questionPairStage4.map(lambda x:(x[0],x[1][0]+":"+x[1][1]+":"+x[1][2]+":"+x[1][3]+":"+x[1][4]+":"+x[1][5]+":"+x[1][6]) )
print(questionPairStage5.take(5))

[('8443560', 'thrift=0.49115149799457625:cassandra=0.33217741418255603:codeplex=0.21631640020847592:protocol=0.17369330327033117:len=0.16851337078099782:host=0.1655789277138693:cluster=0.13966007860011193'), ('32109090', 'interacting=0.3553307011283048:viewcontroller=0.32994375674521975:pushnotification=0.2567311744813316:navigation=0.21832888292463984:notification=0.1391414077733566:depending=0.12887402255724206:g=0'), ('12657390', 'paragraph=0.5987292905603192:tutorial=0.3434025110838559:knowing=0.2798768873942756:format=0.24865331108614458:users=0.23617071563762812:f=0:g=0'), ('13198270', 'webmethod=0.28331548275147156:orgdata=0.2754159451684192:classname=0.2676531474462438:packagename=0.2533064402056847:javax=0.18742974309228122:integer=0.17559729242900116:executions=0.1615754361040162'), ('13409430', 'viewport=0.6842902201655965:floating=0.5905586792866232:component=0.24216553928531787:d=0:e=0:f=0:g=0')]


In [23]:
questionPairStage6=questionPairStage5.union(tag_1000)
def f6_1(a,b):
    return a+":"+b
def f6_2(a,b):
    return a+b
questionPairStage6=questionPairStage6.aggregateByKey("",f6_1,f6_2).map(lambda x: (x[0], x[1][1:]))

[('12987940', 'makefile=1000:sublimetext=1000:sublimetext=1000:sublime=0.4775339837572034:recognizes=0.1949336733023698:configure=0.19467505473473867:build=0.16882855244737976'), ('13113460', 'java=1000:multithreading=1000:producerconsumer=0.43347053831357946:rnd=0.2962562193237151:howcome=0.17938674837579147:severe=0.17172818179232274:runnable=0.1470669108569522')]


In [40]:
print(questionPairStage6.take(10))

[('12987940', 'sublime=0.4775339837572034:recognizes=0.1949336733023698:configure=0.19467505473473867:build=0.16882855244737976:compress=0.16410336766846545:makefile=0.16061359792522345:upload=0.107848335320749:makefile=1000:sublimetext=1000:sublimetext=1000'), ('13113460', 'producerconsumer=0.43347053831357946:rnd=0.2962562193237151:howcome=0.17938674837579147:severe=0.17172818179232274:runnable=0.1470669108569522:thread=0.1463882510034713:followings=0.13457587551302952:java=1000:multithreading=1000'), ('25967490', 'wysiwyg=0.2859475597530611:em=0.23585097652903844:hreflang=0.2324039220977838:responsive=0.2055281493790694:raw=0.17739762857658053:forgot=0.13064120122839845:css=0.11536843705426075:responsivedesign=1000:website=1000:wysiwyg=1000:mobilewebsite=1000'), ('16224290', 'tablesorter=0.3728385962074199:opportunity=0.2820138920357604:opportunities=0.21715077980963665:tr=0.13971821645250038:script=0.11655287166457691:js=0.11409780482561128:docs=0.09773735753788292:python=1000:html

In [ ]:
questionKeywordAndTag = questionPairStage6.flatMapValues(lambda x: x.split(":")).filter(lambda x: float(x[1].split("=")[1]) > 0)
questionKeywordAndTag = questionKeywordAndTag.map(lambda x: (x[0], (x[1].split("=")[0], x[1].split("=")[1])))
print(questionKeywordAndTag.take(5))

In [ ]:
partitionNum = 100
def hashKeyword(x):
    return (int(hashlib.sha1(x.encode()).hexdigest(), 16) % (10 ** 8)) % partitionNum

questionHashedKeywordAndTag = questionKeywordAndTag.map(lambda x: (x[0], (hashKeyword(x[1][0]), x[1][1])))
print(questionHashedKeywordAndTag.take(5))

In [ ]:
questionPartitions = questionHashedKeywordAndTag.map(lambda x: (x[1][0], x[0]))
questionPartitions = questionPartitions.groupByKey().sortBy(lambda x: x[0]).map(lambda x: (x[0], set(x[1])))
print(questionPartitions.count())

In [ ]:
newQuestion = sc.wholeTextFiles("new_q.txt")
newQuestion = newQuestion.map(lambda x: re.sub('\n', ' ', x[1]))
newQuestion = newQuestion.map(lambda x: re.sub('<.*?>', ' ', x.lower()))
newQuestion = newQuestion.map(lambda x: re.sub('[^a-zA-Z0-9\']+', ' ', x))

newQuestionKeywordsStageOne = newQuestion.flatMap(lambda x: x.lower().split(" ")).filter(lambda x: x != '' and x not in stopWordsList).map(lambda x: (x, 1)).reduceByKey(add).map(lambda x: (x[0], 'new=' + str(x[1])))
print(newQuestionKeywordsStageOne.count())

newQuestionKeywordsStageTwoMap = newQuestionKeywordsStageOne.union(questionPairStage2MapPhase).countByKey()
newQuestionKeywordsStageTwo = newQuestionKeywordsStageOne.map(lambda x: (x[0], (1 + np.log(int(x[1].split("=")[1]))) * np.log((N + 1)/newQuestionKeywordsStageTwoMap.get(x[0]))))
print(newQuestionKeywordsStageTwo.count())

newQuestionKeywordsStageThreeSquare = newQuestionKeywordsStageTwo.map(lambda x: np.square(x[1])).reduce(add)
newQuestionKeywordsStageThree = newQuestionKeywordsStageTwo.map(lambda x: (x[0], x[1]/np.sqrt(newQuestionKeywordsStageThreeSquare)))
print(newQuestionKeywordsStageThree.take(5))

keywordAndTagSet = keywordset.union(tagLower.values().collect())
newQuestionKeywordsStageFour = newQuestionKeywordsStageThree.filter(lambda x: x[0] in keywordAndTagSet).sortBy(lambda x: -x[1])
print(newQuestionKeywordsStageFour.take(5))

In [ ]:
keywordNum = 10
newQuestionKeywords = newQuestionKeywordsStageFour.zipWithIndex().filter(lambda x: x[1] < keywordNum).keys()
print(newQuestionKeywords.take(keywordNum))

In [ ]:
newQuestionHashedKeywords = newQuestionKeywords.map(lambda x: hashKeyword(x[0])).collect()
print(newQuestionHashedKeywords)
bucketQuestions = questionPartitions.filter(lambda x: x[0] in newQuestionHashedKeywords).map(lambda x: x[1]).reduce(lambda a, b: a.union(b))
print(len(bucketQuestions))

In [ ]:
# union new question to similar question
candidateSimilarQuestions = questionLower.filter(lambda x: x[0] in bucketQuestions)
candidateSimilarQuestions = newQuestion.map(lambda x: ['0', x]).union(candidateSimilarQuestions)
candidateSimilarQuestions = candidateSimilarQuestions.map(lambda x: (x[0], list(filter(None, x[1].split(" ")))))
print(candidateSimilarQuestions.count())

# generate shingles
shingleLength = 2
candidateQuestionShingles = candidateSimilarQuestions.map(lambda x: (x[0], [" ".join(x[1][i: i + shingleLength]) for i in range(len(x[1]) - shingleLength + 1)]))
print(candidateQuestionShingles.take(1))

# hash shingles
candidateQuestionHashedShingles = candidateQuestionShingles.map(lambda x: (x[0], [hashlib.sha1(x[1][i].encode()).hexdigest() for i in range(len(x[1]) - 1)]))
print(candidateQuestionHashedShingles.take(1))

# minhash
numPerm = 128
m = {}
def minhash(x):
    m[x[1]] = MinHash(num_perm = numPerm)
    for shingle in x[0][1]:
        m[x[1]].update(shingle.encode('utf8'))
    return (x[0][0], m[x[1]])
candidateQuestionMinhash = candidateQuestionHashedShingles.zipWithIndex().map(minhash)

newQuestionMinhash = candidateQuestionMinhash.take(1)[0][1]
candidateQuestionMinhash = candidateQuestionMinhash.filter(lambda x: x[0] != '0')

# lsh
threshold = 0.1
lsh = MinHashLSH(threshold = threshold, num_perm = numPerm)
with lsh.insertion_session() as session:
    for x in candidateQuestionMinhash.collect():
        session.insert(x[0], x[1])
similarQuestions = lsh.query(newQuestionMinhash)
print(len(similarQuestions))

In [ ]:
sqlContext = SQLContext(sc)

questionDataFrame = sqlContext.read.format("csv").options(header="true").load("test_q.csv")
answerDataFrame = sqlContext.read.format("csv").options(header="true").load("test_a.csv")
tagDataFrame = sqlContext.read.format("csv").options(header="true").load("Tags.csv")

print("finish reading source files")

In [ ]:
similarQuestionMinhash = candidateQuestionMinhash.filter(lambda x: x[0] in similarQuestions)
similarQuestionJaccardSimilarity = similarQuestionMinhash.map(lambda x: (x[0], newQuestionMinhash.jaccard(x[1]))).collectAsMap()

similarQuestionDataFrame = questionDataFrame.filter(questionDataFrame.Id.isin(similarQuestions))
similarQuestionDataFrame = similarQuestionDataFrame.join(answerDataFrame, questionDataFrame.Id == answerDataFrame.ParentId).select(questionDataFrame.Id, questionDataFrame.Body, answerDataFrame.Body, answerDataFrame.Score)
print(similarQuestionDataFrame.take(1))

similarQuestionAnswers = similarQuestionDataFrame.rdd.map(lambda x: (x[0], ((x[1].replace("\"\"", "\'").replace("\"", ""), x[2].replace("\"\"", "\'").replace("\"", "")), int(x[3]))))
similarQuestionAnswerScoreSqrt = similarQuestionAnswers.map(lambda x: (x[0], np.square(x[1][1]))).reduceByKey(add).map(lambda x:(x[0], np.sqrt(x[1])))
similarQuestionAnswerScoreSqrt = similarQuestionAnswerScoreSqrt.collectAsMap()

similarQuestionAnswerScoreNorm = similarQuestionAnswers.map(lambda x: (x[0], (x[1][0], x[1][1]/similarQuestionAnswerScoreSqrt.get(x[0]))))
similarQuestionAnswerScoreNorm = similarQuestionAnswerScoreNorm.map(lambda x: (x[0], (x[1][0], x[1][1] * similarQuestionJaccardSimilarity.get(x[0]))))
print(similarQuestionAnswerScoreNorm.take(1))

In [ ]:
answerCount = 10
similarQuestionAnswerSorted = similarQuestionAnswerScoreNorm.map(lambda x: x[1]).sortBy(lambda x: -x[1])
similarQuestionAnswerTop = similarQuestionAnswerSorted.take(answerCount)

In [ ]:
with open('suggested_answers.csv', 'w') as suggestedAnswers:
        writer = csv.writer(suggestedAnswers, delimiter=',')
        writer.writerow(['Rank', 'Original Question', 'Answer'])
        for idx, questionAnswer in enumerate(similarQuestionAnswerTop):
            writer.writerow([idx + 1, questionAnswer[0][0], questionAnswer[0][1]])
            
for idx, questionAnswer in enumerate(similarQuestionAnswerTop):
    print("Rank: " + str(idx + 1))
    print("Original Question: " + questionAnswer[0][0])
    print("Answer: " + questionAnswer[0][1])
    print()

In [42]:
#count top 10 keyword:by tag and by word
#(word) title+body->upvotes
#(k-items) title+body->upvotes

1


In [ ]:
#count top 10 keyword by tag

In [23]:
tagWords=tagLower.map(lambda x:(x[1],1))
tagCounts=tagWords.reduceByKey(add)
tagCounts.take(10)

[('flex', 3307),
 ('actionscript', 5112),
 ('svn', 2361),
 ('tortoisesvn', 441),
 ('sitemap', 157),
 ('algorithm', 6837),
 ('colors', 1257),
 ('colorspace', 26),
 ('c', 178780),
 ('scripting', 1030)]

In [27]:
tagSorted = tagCounts.sortBy(lambda x: -x[1])
tagSorted.take(100)

[('c', 178780),
 ('javascript', 124156),
 ('java', 116276),
 ('php', 99151),
 ('android', 90795),
 ('jquery', 78598),
 ('python', 71010),
 ('html', 68561),
 ('ios', 53392),
 ('css', 48060),
 ('mysql', 42603),
 ('rubyonrails', 36174),
 ('sql', 35782),
 ('aspnet', 30390),
 ('objectivec', 27040),
 ('sqlserver', 26494),
 ('net', 26104),
 ('aspnetmvc', 23932),
 ('iphone', 21680),
 ('angularjs', 20367),
 ('arrays', 19799),
 ('visualstudio', 18035),
 ('json', 17669),
 ('ruby', 17247),
 ('r', 15708),
 ('ajax', 15629),
 ('regex', 15349),
 ('xml', 14768),
 ('nodejs', 14525),
 ('xcode', 13630),
 ('linux', 13395),
 ('windows', 13042),
 ('swift', 13037),
 ('django', 13002),
 ('wpf', 12400),
 ('database', 11860),
 ('excel', 11390),
 ('vbnet', 10453),
 ('string', 10427),
 ('spring', 10041),
 ('wordpress', 9922),
 ('eclipse', 9919),
 ('multithreading', 9006),
 ('twitterbootstrap', 8882),
 ('laravel', 8858),
 ('entityframework', 8230),
 ('oracle', 7647),
 ('git', 7626),
 ('facebook', 7598),
 ('forms', 

In [32]:
#count top 10 keyword by words
questionWords=questionPairStage1.map(lambda x:(x[0].split("@")[1],1))
questionCounts=questionWords.reduceByKey(add)
questionCounts.take(10)
print(questionCounts.take(100))

[('letter', 6238), ('pass', 45795), ('calls', 24585), ('desktop', 13499), ('settings', 25407), ('vb', 8917), ('documentation', 28299), ('compiled', 9183), ('locally', 6986), ('components', 10922), ('basis', 2781), ('benefits', 1396), ('wikis', 75), ('redirection', 1695), ('loads', 13298), ('setbounds', 1070), ('tr', 22725), ('interact', 2705), ('pointer', 16022), ('timertask', 480), ('messagebox', 4422), ('commons', 3175), ('depth', 3957), ('permitted', 1075), ('pop', 8754), ('checker', 888), ('comments', 12463), ('elastic', 1409), ('loss', 3484), ('members', 6829), ('published', 3417), ('score', 5380), ('xxx', 7578), ('news', 4478), ('settimeout', 4565), ('exceptions', 7678), ('rufus', 42), ('identity', 7428), ('opacity', 5013), ('xmltextreader', 153), ('edited', 6364), ('originate', 159), ('lite', 835), ('mydiv', 518), ('walls', 436), ('inactive', 1429), ('wallpaper', 451), ('addproduct', 160), ('longitude', 4785), ('uint', 4644), ('abs', 2747), ('visit', 4373), ('fin', 989), ('stati

In [33]:
questionSorted = questionCounts.sortBy(lambda x: -x[1])
questionSorted.take(100)

[('class', 272436),
 ('application', 150735),
 ('int', 137590),
 ('list', 131790),
 ('image', 117832),
 ('null', 114421),
 ('java', 105866),
 ('test', 104412),
 ('view', 103943),
 ('php', 103911),
 ('array', 100449),
 ('number', 95158),
 ('button', 95047),
 ('script', 94101),
 ('li', 93626),
 ('files', 92349),
 ('query', 89745),
 ('project', 89285),
 ('click', 86513),
 ('div', 86133),
 ('stack', 85460),
 ('display', 84367),
 ('output', 84007),
 ('web', 82988),
 ('post', 82852),
 ('javascript', 77513),
 ('log', 77283),
 ('request', 74278),
 ('key', 71668),
 ('load', 71314),
 ('link', 70505),
 ('title', 69360),
 ('path', 68573),
 ('jquery', 68138),
 ('row', 66932),
 ('length', 66586),
 ('height', 66175),
 ('current', 64607),
 ('program', 64423),
 ('order', 64285),
 ('sql', 64177),
 ('users', 63465),
 ('multiple', 63454),
 ('blockquote', 63305),
 ('source', 62652),
 ('xml', 62363),
 ('variable', 61438),
 ('item', 60055),
 ('column', 59650),
 ('print', 59081),
 ('correct', 58953),
 ('event

In [39]:
#count top 10 keyword by keywords
def fmap2(x):
    outputs=[]
    for i in range(num_keywords):
        y=x[1][i].split("=")
        if float(y[1])>0.4:
           outputs.append((y[0],1))
    return (outputs)
keywordWords=questionPairStage4.flatMap(fmap2).reduceByKey(add)
keywordWords.take(10)

[('getc', 40),
 ('visit', 108),
 ('recognizing', 21),
 ('squares', 98),
 ('hdc', 92),
 ('published', 120),
 ('desktop', 220),
 ('payroll', 39),
 ('abs', 105),
 ('interact', 64)]

In [40]:
keywordSorted = keywordWords.sortBy(lambda x: -x[1])
keywordSorted.take(100)

[('td', 2058),
 ('facebook', 1847),
 ('foo', 1697),
 ('iframe', 1491),
 ('pdf', 1440),
 ('git', 1316),
 ('arr', 1260),
 ('regex', 1253),
 ('csv', 1142),
 ('xsl', 1090),
 ('webview', 1075),
 ('video', 1074),
 ('sqlite', 1069),
 ('textbox', 1010),
 ('curl', 1008),
 ('matrix', 992),
 ('checkbox', 987),
 ('intent', 966),
 ('col', 961),
 ('azure', 958),
 ('modal', 949),
 ('excel', 933),
 ('canvas', 931),
 ('radio', 922),
 ('eclipse', 919),
 ('word', 916),
 ('socket', 915),
 ('div', 905),
 ('dll', 904),
 ('dialog', 903),
 ('tab', 900),
 ('myclass', 895),
 ('cell', 892),
 ('notification', 885),
 ('vector', 885),
 ('session', 871),
 ('listview', 869),
 ('xcode', 865),
 ('span', 864),
 ('sharepoint', 864),
 ('sum', 861),
 ('svn', 859),
 ('django', 841),
 ('svg', 830),
 ('slider', 826),
 ('combobox', 823),
 ('gridview', 820),
 ('cookie', 817),
 ('abc', 815),
 ('perl', 811),
 ('array', 811),
 ('matlab', 804),
 ('dictionary', 802),
 ('meteor', 801),
 ('iphone', 798),
 ('qt', 798),
 ('magento', 795

In [36]:
print(1)

1


In [1]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
from pyspark.sql.functions import col
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import IndexToString
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import HashingTF, IDF

In [2]:
data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('Questions.csv')

In [3]:
dataSample = data.sample(False,0.10,80)
print(dataSample.take(2))

[Row(Id=1010, OwnerUserId=67, CreationDate=datetime.datetime(2008, 8, 4, 11, 59, 42), ClosedDate=None, Score=14, Title='How to get the value of built; encoded ViewState?', Body='"<p>I need to grab the base64-encoded representation of the ViewState. Obviously this would not be available until fairly late in the request lifecycle; which is OK.</p>  <p>For example; if the output of the page includes:</p>  <pre><code>&lt;input type=""hidden"" name=""__VIEWSTATE""    id=""__VIEWSTATE"" value=""/wEPDwUJODU0Njc5MD...=="" /&gt; </code></pre>  <p>I need a way on the server side to get the value <code>""/wEPDwUJODU0Njc5MD...==""</code></p>  <p>To clarify; I need this value when the page is being rendered; not on PostBack. e.g. I need to know the ViewState value that is being <em>sent</em> to the client; not the ViewState I\'m getting back from them.</p> "', Code=True), Row(Id=1610, OwnerUserId=328, CreationDate=datetime.datetime(2008, 8, 5, 5, 37, 31), ClosedDate=None, Score=63, Title='Can I log

In [4]:
drop_list = ['Id','Title', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Code']
dataInputs = dataSample.select([column for column in data.columns if column not in drop_list])
dataInputs.show(5)
dataInputs.printSchema()

+-----+--------------------+
|Score|                Body|
+-----+--------------------+
|   14|"<p>I need to gra...|
|   63|"<p>If I'm adding...|
|   51|<p>Getting back i...|
|   12|<p>Let's say that...|
|   14|"<p>I am getting ...|
+-----+--------------------+
only showing top 5 rows

root
 |-- Score: integer (nullable = true)
 |-- Body: string (nullable = true)



In [5]:
name = 'Score'
def labelfunction(x):
    if (int(x)>0):
        return "yes"
    else: return "no"
udf = UserDefinedFunction(labelfunction, StringType())
new_df = dataInputs.select(*[udf(column).alias(name) if column == name else column for column in dataInputs.columns])

In [6]:
new_df = new_df.filter(new_df.Score.isNotNull())
#new_df = new_df.filter(new_df.Title.isNotNull())
new_df = new_df.filter(new_df.Body.isNotNull())
new_df.show(1000)

+-----+--------------------+
|Score|                Body|
+-----+--------------------+
|  yes|"<p>I need to gra...|
|  yes|"<p>If I'm adding...|
|  yes|<p>Getting back i...|
|  yes|<p>Let's say that...|
|  yes|"<p>I am getting ...|
|  yes|"<p>I'm pretty ne...|
|  yes|"<p>I've been han...|
|  yes|<p>I'm suddenly b...|
|  yes|<p>I am writing a...|
|  yes|<p>Say you have a...|
|  yes|"<p>This is a nas...|
|  yes|<p>I'm always loo...|
|  yes|<p>I'm new to dev...|
|  yes|"<p>While trying ...|
|  yes|"<p>I took a look...|
|  yes|"<p>First off I u...|
|  yes|<p>I was writing ...|
|  yes|<p>How do you run...|
|  yes|"<p>We have a sit...|
|  yes|<p>I'm a complete...|
|  yes|<p>I seem to be m...|
|  yes|"<blockquote>   <...|
|  yes|<p>We have a situ...|
|  yes|"<p>I'm using Int...|
|  yes|<p>I was curious ...|
|  yes|"<p>I want to see...|
|  yes|"<p>FCKeditor has...|
|  yes|<p>I'm pretty kee...|
|   no|<p>Over the last ...|
|  yes|"<p>Is there a wa...|
|  yes|"<p>We often hear...|
|  yes|<p>We a

In [45]:
#max_value = new_df.agg({"Score": "max"}).collect()[0][0]
#new_df=new_df.withColumn("Score", new_df["Score"].cast(IntegerType()))
#print(max_value)

9


In [13]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stopWordsList)
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=1000000, minDF=50)
label_stringIdx = StringIndexer(inputCol = "Score", outputCol = "label")

In [14]:

#label_stringIdx = IndexToString( inputCol="Score", outputCol="Label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
#pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors])
pipelineFit = pipeline.fit(new_df)
dataset = pipelineFit.transform(new_df)
dataset.show(500)

+-----+--------------------+--------------------+--------------------+--------------------+-----+
|Score|                Body|               words|            filtered|            features|label|
+-----+--------------------+--------------------+--------------------+--------------------+-----+
|  yes|"<p>I need to gra...|[p, i, need, to, ...|[grab, base64, en...|(9253,[0,1,2,3,26...|  1.0|
|  yes|"<p>If I'm adding...|[p, if, i, m, add...|[adding, column, ...|(9253,[43,44,46,1...|  1.0|
|  yes|<p>Getting back i...|[p, getting, back...|[bit, net, years,...|(9253,[64,102,104...|  1.0|
|  yes|<p>Let's say that...|[p, let, s, say, ...|[argb, color, pre...|(9253,[0,3,39,69,...|  1.0|
|  yes|"<p>I am getting ...|[p, i, am, gettin...|[error, blockquot...|(9253,[0,1,2,3,4,...|  1.0|
|  yes|"<p>I'm pretty ne...|[p, i, m, pretty,...|[pretty, company,...|(9253,[0,6,7,14,1...|  1.0|
|  yes|"<p>I've been han...|[p, i, ve, been, ...|[ve, handed, tabl...|(9253,[0,3,5,7,9,...|  1.0|
|  yes|<p>I'm sudden

In [15]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 1)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))
trainingData.take(1)

Training Dataset Count: 88402
Test Dataset Count: 37891


[Row(Score='no', Body='"<blockquote>   <p><strong>Possible Duplicate:</strong><br>   <a href=""http://stackoverflow.com/questions/1005539/datagridview-sort-programmitically"">DataGridView Sort Programmitically</a>  </p> </blockquote>    <p>How Sort Datagridview ?</p> "', words=['blockquote', 'p', 'strong', 'possible', 'duplicate', 'strong', 'br', 'a', 'href', 'http', 'stackoverflow', 'com', 'questions', '1005539', 'datagridview', 'sort', 'programmitically', 'datagridview', 'sort', 'programmitically', 'a', 'p', 'blockquote', 'p', 'how', 'sort', 'datagridview', 'p'], filtered=['blockquote', 'strong', 'duplicate', 'strong', 'br', 'href', 'http', 'stackoverflow', 'questions', '1005539', 'datagridview', 'sort', 'programmitically', 'datagridview', 'sort', 'programmitically', 'blockquote', 'sort', 'datagridview'], features=SparseVector(9253, {8: 1.0, 9: 2.0, 24: 1.0, 68: 1.0, 111: 2.0, 217: 1.0, 278: 1.0, 530: 3.0, 1162: 1.0, 3678: 3.0}), label=0.0)]

In [16]:
lr = LogisticRegression(maxIter=100, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 1000, truncate = 30)

+------------------------------+-----+------------------------------+-----+----------+
|                          Body|Score|                   probability|label|prediction|
+------------------------------+-----+------------------------------+-----+----------+
|"<p>Is it possible to use <...|   no|[0.4999995523154908,0.50000...|  0.0|       1.0|
|"<p>this is the script im u...|  yes|[0.4999992700321526,0.50000...|  1.0|       1.0|
|"<p>I'm using jquery to go ...|  yes|[0.4999979607444501,0.50000...|  1.0|       1.0|
|"<p>I write a simple web se...|   no|[0.49999771465625786,0.5000...|  0.0|       1.0|
|"<p>I have an open-source l...|  yes|[0.4999871128657412,0.50001...|  1.0|       1.0|
|"<p>Reference to : <a href=...|  yes|[0.4999809451835547,0.50001...|  1.0|       1.0|
|<p>I've got a legacy ASP we...|   no|[0.4999808223069684,0.50001...|  0.0|       1.0|
|<p>I need to merge this ind...|  yes|[0.4999696842216995,0.50003...|  1.0|       1.0|
|"<p>I have a glassfish clus...|   no|[0.49

In [17]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.5581593472055252

In [ ]:
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['label'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 1000, truncate = 30)

In [115]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.5552636058194875

In [19]:
lr = LogisticRegression(maxIter=20, regParam=0.5, elasticNetParam=0.8)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 200, truncate = 30)

+------------------------------+-----+------------------------------+-----+----------+
|                          Body|Score|                   probability|label|prediction|
+------------------------------+-----+------------------------------+-----+----------+
|"<p>I am trying to log into...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|"<p>Suppose that I have a r...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|"<p>My main goal: Collect s...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|"<p>how to add unknow numbe...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|"<blockquote>   <p><strong>...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|<p>I need to create a web s...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|<p>My tinyMCE editor is sho...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|"<p>How can I draw a simple...|   -1|[0.926738165391712,0.062209...|  1.0|       0.0|
|"<p>How would I change the ...|   -1|[0.92

In [20]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.888564349616851

In [152]:
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 200, truncate = 30)

+------------------------------+-----+------------------------------+-----+----------+
|                          Body|Score|                   probability|label|prediction|
+------------------------------+-----+------------------------------+-----+----------+
|"<p>In answering <a href=""...|  yes|[0.4999480901160706,0.50005...|  1.0|       1.0|
|"<p>Sorry if this a repeat ...|   no|[0.49984049413069437,0.5001...|  0.0|       1.0|
|"<p>So I asked another rela...|  yes|[0.4997395843152862,0.50026...|  1.0|       1.0|
|"<p>I have learned a Machin...|  yes|[0.4994855167611214,0.50051...|  1.0|       1.0|
|"<p>I'm planning on gaining...|   no|[0.499435407116347,0.500564...|  0.0|       1.0|
|"<p>I just read <a href=""h...|  yes|[0.49939374383043095,0.5006...|  1.0|       1.0|
|"<p>I am trying to set depe...|   no|[0.4993485578108331,0.50065...|  0.0|       1.0|
|"<p>I'm attempting to use g...|  yes|[0.4992843651085154,0.50071...|  1.0|       1.0|
|"<h3>Technical circumstance...|  yes|[0.49

In [153]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.36940530125627613

In [163]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1000000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])

In [164]:
pipelineFit = pipeline.fit(new_df)
dataset = pipelineFit.transform(new_df)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))
trainingData.take(1)

Training Dataset Count: 88721
Test Dataset Count: 37572


[Row(Score='no', Body='"<blockquote>   <p><strong>Possible Duplicate:</strong><br>   <a href=""http://stackoverflow.com/questions/1005539/datagridview-sort-programmitically"">DataGridView Sort Programmitically</a>  </p> </blockquote>    <p>How Sort Datagridview ?</p> "', words=['blockquote', 'p', 'strong', 'possible', 'duplicate', 'strong', 'br', 'a', 'href', 'http', 'stackoverflow', 'com', 'questions', '1005539', 'datagridview', 'sort', 'programmitically', 'datagridview', 'sort', 'programmitically', 'a', 'p', 'blockquote', 'p', 'how', 'sort', 'datagridview', 'p'], filtered=['blockquote', 'strong', 'duplicate', 'strong', 'br', 'href', 'http', 'stackoverflow', 'questions', '1005539', 'datagridview', 'sort', 'programmitically', 'datagridview', 'sort', 'programmitically', 'blockquote', 'sort', 'datagridview'], rawFeatures=SparseVector(1000000, {254119: 1.0, 265432: 1.0, 325720: 3.0, 375958: 1.0, 388482: 1.0, 555675: 2.0, 630665: 1.0, 650892: 3.0, 720727: 1.0, 858475: 1.0, 928407: 2.0, 986

In [167]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 1) \
    .select("Body","Score","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 200, truncate = 30)

Py4JJavaError: An error occurred while calling o8300.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 2024.0 failed 1 times, most recent failure: Lost task 2.0 in stage 2024.0 (TID 19079, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:1930)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1561)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2278)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2202)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2060)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1567)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2278)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2202)
	at java.io.ObjectInputStream.readOrdinaryObject(Obj

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 65149)
Traceback (most recent call last):
  File "/Users/Sisyphus/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/Sisyphus/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/Sisyphus/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/Sisyphus/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
-------------

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)